In [1]:
import glob
import io
import math
import time
import os

import keras.backend as K
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from PIL import Image
from keras import Sequential, Input, Model
# from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import ReLU
from keras.layers import Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing import image
from scipy.misc import imread, imsave
from scipy.stats import entropy

# K.set_image_dim_ordering('tf')

# np.random.seed(1337)

def write_log(callback, name, loss, batch_no):
    """
    Write training summary to TensorBoard
    """
    summary = tf.Summary()
    summary_value = summary.value.add()
    summary_value.simple_value = loss
    summary_value.tag = name
    callback.writer.add_summary(summary, batch_no)
    callback.writer.flush()

    
    
def build_generator():
    gen_model = Sequential()

    gen_model.add(Dense(input_dim=100, output_dim=2048))
    gen_model.add(LeakyReLU(alpha=0.2))



    gen_model.add(Dense(256 * 8 * 8))
    gen_model.add(BatchNormalization())
    gen_model.add(LeakyReLU(alpha=0.2))


    # 8x8 
    gen_model.add(Reshape((8, 8, 256), input_shape=(256 * 8 * 8,)))
    gen_model.add(UpSampling2D(size=(2, 2)))

    # 16x16
    gen_model.add(Conv2D(128, (5, 5), padding='same'))
    gen_model.add(LeakyReLU(alpha=0.2))

    gen_model.add(UpSampling2D(size=(2, 2)))

    # 32x32
    gen_model.add(Conv2D(64, (5, 5), padding='same'))
    gen_model.add(LeakyReLU(alpha=0.2))

    gen_model.add(UpSampling2D(size=(2, 2)))


    # 64x64
    gen_model.add(Conv2D(32, (5, 5), padding='same'))
    gen_model.add(LeakyReLU(alpha=0.2))

    gen_model.add(UpSampling2D(size=(2, 2)))


    # 128x128
    gen_model.add(Conv2D(3, (5, 5), padding='same'))
    gen_model.add(LeakyReLU(alpha=0.2))


    return gen_model


def build_discriminator():


    dis_model = Sequential()

    w = 128
    h = 128
    dis_model.add(
        Conv2D(128, (5, 5),
               padding='same',
               input_shape=(w, h, 3))
    )


    dis_model.add(LeakyReLU(alpha=0.2))
    dis_model.add(MaxPooling2D(pool_size=(2, 2)))

    # 64x64
    dis_model.add(Conv2D(256, (3, 3)))
    dis_model.add(LeakyReLU(alpha=0.2))
    dis_model.add(MaxPooling2D(pool_size=(2, 2)))

    # 32x32
    dis_model.add(Conv2D(512, (3, 3)))
    dis_model.add(LeakyReLU(alpha=0.2))
    dis_model.add(MaxPooling2D(pool_size=(2, 2)))

    # 16x16
    dis_model.add(Flatten())
    dis_model.add(Dense(1024))
    dis_model.add(LeakyReLU(alpha=0.2))

    dis_model.add(Dense(1))
    dis_model.add(Activation('sigmoid'))

    return dis_model




rawImagesPath = "rawimages"
readyImagesPath = "readyimages"

def pre_process_img(input_path, output_path):

    w = 128
    h = 128
    files = os.listdir(rawImagesPath)
    os.chdir(input_path)
    if(not os.path.exists(output_path)):
        os.makedirs(output_path)
    for file in files:
        if(os.path.isfile(file) & file.endswith('.jpg')):
            img = Image.open(file)
            img = img.resize((w, h), Image.ANTIALIAS)
            img.save(os.path.join(readyImagesPath,file))



def post_process_img(img):
    res = img
    return res

def train():
    batch_size = 64
    z_shape = 100
    epochs = 1000
    dis_learning_rate = 0.0005
    gen_learning_rate = 0.0005
    dis_momentum = 0.9
    gen_momentum = 0.9
    dis_nesterov = True
    gen_nesterov = True


    # Loading images
    all_images = []
    for index, filename in enumerate(glob.glob(readyImagesPath + '/*.jpg')): 
        image = imread(filename, flatten=False, mode='RGB')
        all_images.append(image)

    # Convert to Numpy ndarray
    X = np.array(all_images)
    X = (X - 127.5) / 127.5


    # Define optimizers
    dis_optimizer = SGD(lr=dis_learning_rate, momentum=dis_momentum, nesterov=dis_nesterov)
    gen_optimizer = SGD(lr=gen_learning_rate, momentum=gen_momentum, nesterov=gen_nesterov)


    gen_model = build_generator()
    gen_model.compile(loss='binary_crossentropy', optimizer=gen_optimizer)


    dis_model = build_discriminator()
    dis_model.compile(loss='binary_crossentropy', optimizer=dis_optimizer)

    
    
    
    ####checkpoint =====================

    checkpoint_dir = './training_checkpoints'
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
    checkpoint = tf.train.Checkpoint(generator_optimizer     = gen_optimizer,
                                     discriminator_optimizer = dis_optimizer,
                                     generator               = gen_model,
                                     discriminator           = dis_model)

    #=======================================
    
    
    adversarial_model = Sequential()
    adversarial_model.add(gen_model)
    dis_model.trainable = False
    adversarial_model.add(dis_model)


    adversarial_model.compile(loss='binary_crossentropy', optimizer=gen_optimizer)

    tensorboard = TensorBoard(log_dir="logs/{}".format(time.time()), write_images=True, write_grads=True,
                                  write_graph=True)

    tensorboard.set_model(gen_model)
    tensorboard.set_model(dis_model)


    for epoch in range(epochs):
        print("Epoch is", epoch)
        number_of_batches = int(X.shape[0] / batch_size)
        print("Number of batches", number_of_batches)
        for index in range(number_of_batches):


            z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))


            image_batch = X[index * batch_size:(index + 1) * batch_size]

            generated_images = gen_model.predict_on_batch(z_noise)

            y_real = np.ones(batch_size) - np.random.random_sample(batch_size) * 0.2
            y_fake = np.random.random_sample(batch_size) * 0.2

            dis_loss_real = dis_model.train_on_batch(image_batch, y_real)
            dis_loss_fake = dis_model.train_on_batch(generated_images, y_fake)
            d_loss = (dis_loss_real+dis_loss_fake)/2
            print("d_loss:", d_loss)


            z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))
            g_loss = adversarial_model.train_on_batch(z_noise, [1] * batch_size)


            print("g_loss:", g_loss)

            """
            Save losses to Tensorboard after each epoch
            """
            write_log(tensorboard, 'discriminator_loss', np.mean(d_loss), epoch)
            write_log(tensorboard, 'generator_loss', np.mean(g_loss), epoch)
            
            
            
            
            
            if epoch % 10 == 0:
                
                
                #save models
                checkpoint.save(file_prefix = checkpoint_prefix)

                z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))
                gen_images1 = gen_model.predict_on_batch(z_noise)
                index = 0
                for img in gen_images1[:2]:
#                     save_rgb_img(img, "results/one_{}.jpg".format(epoch))
                    
                    imsave('results/img_{}_'.format(epoch) + str(index) + '.jpg',img)
                    index += 1




    # Specify the path for the generator model
    gen_model.save("./gen_model.h5") 

    # Specify the path for the discriminator model
    dis_model.save("./dis_model.h5") 



Using TensorFlow backend.


In [2]:
train()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:159: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, units=2048)`


Epoch is 0
Number of batches 5


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


d_loss: 0.7052038908004761
g_loss: 0.66679174


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:231: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


...

g_loss: 2.5493855
Epoch is 858
Number of batches 5
d_loss: 0.3798923194408417
g_loss: 2.4132195
d_loss: 0.333448588848114
g_loss: 2.4194012
d_loss: 0.3202032446861267
g_loss: 2.6283689
d_loss: 0.32567644119262695
g_loss: 2.3595262
d_loss: 0.3261866569519043
g_loss: 2.6027448
Epoch is 859
Number of batches 5
d_loss: 0.3590642213821411
g_loss: 2.4178102
d_loss: 0.33378905057907104
g_loss: 2.5383751
d_loss: 0.35441723465919495
g_loss: 2.0872316
d_loss: 0.33123546838760376
g_loss: 2.471736
d_loss: 0.34360891580581665
g_loss: 2.446952
Epoch is 860
Number of batches 5
d_loss: 0.3332946300506592
g_loss: 2.444045
d_loss: 0.3496968448162079
g_loss: 2.3372602
d_loss: 0.360238254070282
g_loss: 2.3831215
d_loss: 0.33072149753570557
g_loss: 2.5700397
d_loss: 0.351332426071167
g_loss: 2.3938146
Epoch is 861
Number of batches 5
d_loss: 0.3725372552871704
g_loss: 2.1271067
d_loss: 0.3693108558654785
g_loss: 2.7542577
d_loss: 0.35951727628707886
g_loss: 2.2327874
d_loss: 0.3204612731933594
g_loss: 2.43

g_loss: 2.4121456
d_loss: 0.337748646736145
g_loss: 2.462083
Epoch is 890
Number of batches 5
d_loss: 0.3474183678627014
g_loss: 2.6133041
d_loss: 0.3419947028160095
g_loss: 2.3382144
d_loss: 0.3329682946205139
g_loss: 2.2953563
d_loss: 0.35184529423713684
g_loss: 2.3897572
d_loss: 0.32640254497528076
g_loss: 2.3839111
Epoch is 891
Number of batches 5
d_loss: 0.35058826208114624
g_loss: 2.248353
d_loss: 0.3378508687019348
g_loss: 2.3040524
d_loss: 0.3391518294811249
g_loss: 2.3687701
d_loss: 0.3460223376750946
g_loss: 2.368418
d_loss: 0.32782942056655884
g_loss: 2.2441983
Epoch is 892
Number of batches 5
d_loss: 0.32536929845809937
g_loss: 2.3061357
d_loss: 0.34840625524520874
g_loss: 2.3671522
d_loss: 0.3552289307117462
g_loss: 2.5415838
d_loss: 0.34952670335769653
g_loss: 2.252224
d_loss: 0.3055208921432495
g_loss: 2.3498185
Epoch is 893
Number of batches 5
d_loss: 0.3392236530780792
g_loss: 2.467572
d_loss: 0.32094573974609375
g_loss: 2.3622575
d_loss: 0.3365318775177002
g_loss: 2.4

d_loss: 0.32772159576416016
g_loss: 2.376432
d_loss: 0.32333904504776
g_loss: 2.5132623
d_loss: 0.32174572348594666
g_loss: 2.4427805
Epoch is 922
Number of batches 5
d_loss: 0.32679781317710876
g_loss: 2.3660855
d_loss: 0.3313802480697632
g_loss: 2.3765922
d_loss: 0.337130606174469
g_loss: 2.6567848
d_loss: 0.33554378151893616
g_loss: 2.4460154
d_loss: 0.33684539794921875
g_loss: 2.3586965
Epoch is 923
Number of batches 5
d_loss: 0.33031290769577026
g_loss: 2.3804433
d_loss: 0.32200729846954346
g_loss: 2.3883212
d_loss: 0.3225960433483124
g_loss: 2.57272
d_loss: 0.3534817099571228
g_loss: 2.16055
d_loss: 0.3462526202201843
g_loss: 2.404015
Epoch is 924
Number of batches 5
d_loss: 0.3406805694103241
g_loss: 2.4136472
d_loss: 0.3383738398551941
g_loss: 2.2680573
d_loss: 0.336362361907959
g_loss: 2.268106
d_loss: 0.33788466453552246
g_loss: 2.6844878
d_loss: 0.3492885231971741
g_loss: 2.2578325
Epoch is 925
Number of batches 5
d_loss: 0.3217514753341675
g_loss: 2.4009163
d_loss: 0.318559

g_loss: 2.366152
d_loss: 0.33297672867774963
g_loss: 2.3218794
d_loss: 0.3392835855484009
g_loss: 2.2666032
d_loss: 0.3218514919281006
g_loss: 2.520703
d_loss: 0.3339378237724304
g_loss: 2.3986778
Epoch is 954
Number of batches 5
d_loss: 0.3480774760246277
g_loss: 2.4038718
d_loss: 0.32904940843582153
g_loss: 2.3140457
d_loss: 0.3168640732765198
g_loss: 2.4336057
d_loss: 0.3307587802410126
g_loss: 2.4473472
d_loss: 0.3333663046360016
g_loss: 2.2164183
Epoch is 955
Number of batches 5
d_loss: 0.3313847780227661
g_loss: 2.3085933
d_loss: 0.32809028029441833
g_loss: 2.3690658
d_loss: 0.3367109000682831
g_loss: 2.29001
d_loss: 0.333313912153244
g_loss: 2.2255096
d_loss: 0.3373647928237915
g_loss: 2.3581967
Epoch is 956
Number of batches 5
d_loss: 0.3214220404624939
g_loss: 2.261119
d_loss: 0.3462815582752228
g_loss: 2.7149105
d_loss: 0.3845667541027069
g_loss: 2.706078
d_loss: 0.3287525773048401
g_loss: 2.6517332
d_loss: 0.31971657276153564
g_loss: 2.5298834
Epoch is 957
Number of batches 

d_loss: 0.3441322445869446
g_loss: 2.2569613
d_loss: 0.3232744336128235
g_loss: 2.6680636
d_loss: 0.3797663450241089
g_loss: 2.285923
d_loss: 0.35241320729255676
g_loss: 2.3911932
d_loss: 0.33629265427589417
g_loss: 2.3913808
Epoch is 986
Number of batches 5
d_loss: 0.3432951867580414
g_loss: 2.3231184
d_loss: 0.33089685440063477
g_loss: 2.131
d_loss: 0.35019922256469727
g_loss: 2.6132576
d_loss: 0.360790878534317
g_loss: 2.561718
d_loss: 0.35009533166885376
g_loss: 2.3404624
Epoch is 987
Number of batches 5
d_loss: 0.33099979162216187
g_loss: 2.4408243
d_loss: 0.32742738723754883
g_loss: 2.6274297
d_loss: 0.35772639513015747
g_loss: 2.3218622
d_loss: 0.3448953628540039
g_loss: 2.2390885
d_loss: 0.3270035982131958
g_loss: 2.4347386
Epoch is 988
Number of batches 5
d_loss: 0.3524106442928314
g_loss: 2.4757705
d_loss: 0.32883840799331665
g_loss: 2.388296
d_loss: 0.33838534355163574
g_loss: 2.3863568
d_loss: 0.3512606620788574
g_loss: 2.2139463
d_loss: 0.3477252721786499
g_loss: 2.64043
E